# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [61]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [62]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-2.39,68,6,4.92,CA,1683839302
1,1,thompson,55.7435,-97.8558,21.55,27,36,4.79,CA,1683839302
2,2,west island,-12.1568,96.8225,28.99,79,75,5.66,CC,1683839302
3,3,rongelap,11.1635,166.8941,27.28,83,94,7.29,MH,1683839302
4,4,vernon,50.2581,-119.2691,24.61,29,70,0.64,CA,1683839302


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lon",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Hummidity",
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot

ImportError: Geographic projection support requires geoviews, pyproj and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Max Temp"] > 30]
city_data_df = city_data_df.loc[city_data_df["Max Temp"] <= 35]
city_data_df = city_data_df.loc[city_data_df["Humidity"]  > 30]
city_data_df = city_data_df.loc[city_data_df["Humidity"] <= 50]
# Drop any rows with null values
 ##city_data_clean_df = city_data_df.dropna()
# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,tindouf,27.6711,-8.1474,31.97,12,28,8.15,DZ,1683839310
40,40,villa emilio carranza,19.9667,-96.6000,30.71,67,13,3.49,MX,1683839310
70,70,i-n-salah,27.1935,2.4607,31.23,13,5,5.22,DZ,1683839315
88,88,coahuayana de hidalgo,18.7000,-103.6583,32.24,50,26,3.91,MX,1683839324
198,198,galliano,29.4422,-90.2992,31.00,66,100,4.12,US,1683839350


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 37 to 576
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     41 non-null     int64  
 1   City        41 non-null     object 
 2   Lat         41 non-null     float64
 3   Lng         41 non-null     float64
 4   Max Temp    41 non-null     float64
 5   Humidity    41 non-null     int64  
 6   Cloudiness  41 non-null     int64  
 7   Wind Speed  41 non-null     float64
 8   Country     41 non-null     object 
 9   Date        41 non-null     int64  
 10  Hotel Name  41 non-null     object 
dtypes: float64(4), int64(4), object(3)
memory usage: 3.8+ KB


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:
radius = 10000
categories = "accommodation.hotel"

limit = 5
params = {
    "categories":categories,
    "limit":limit, 
}
params["filter"] = f"circle:{lon},{lat},{radius}"
params["bias"] = f"proximity:{lon},{lat}"
base_url = "https://api.geoapify.com/v2/places"
name_address = requests.get(f"{base_url}, params={params}&apiKey=970bda399889431fb0e9766578445d0b")
name_address.status_code

401

In [65]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

limit = 5
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df["Lat"]
    lon = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tindouf - nearest hotel: No hotel found
villa emilio carranza - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
galliano - nearest hotel: No hotel found
shendi - nearest hotel: No hotel found
tamiahua - nearest hotel: No hotel found
san luis acatlan - nearest hotel: No hotel found
kokologo - nearest hotel: No hotel found
biltine - nearest hotel: No hotel found
adam - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
pijijiapan - nearest hotel: No hotel found
cleburne - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
samana - nearest hotel: No hotel found
aleg - nearest hotel: No hotel found
timimoun - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
san luis de la loma - n

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,tindouf,27.6711,-8.1474,31.97,12,28,8.15,DZ,1683839310,No hotel found
40,40,villa emilio carranza,19.9667,-96.6000,30.71,67,13,3.49,MX,1683839310,No hotel found
70,70,i-n-salah,27.1935,2.4607,31.23,13,5,5.22,DZ,1683839315,No hotel found
88,88,coahuayana de hidalgo,18.7000,-103.6583,32.24,50,26,3.91,MX,1683839324,No hotel found
198,198,galliano,29.4422,-90.2992,31.00,66,100,4.12,US,1683839350,No hotel found
199,199,shendi,16.6915,33.4341,34.64,8,3,3.75,SD,1683839220,No hotel found
203,203,tamiahua,21.2667,-97.4500,31.73,63,1,5.32,MX,1683839318,No hotel found
227,227,san luis acatlan,16.8167,-98.7333,33.31,45,96,4.95,MX,1683839356,No hotel found
233,233,kokologo,12.1897,-1.8856,35.61,24,95,3.09,BF,1683839358,No hotel found
238,238,biltine,14.5333,20.9167,32.87,28,100,6.38,TD,1683839359,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lon",
    "city",
    "Humidity",
    "Hotel Name",
    "Country",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Hummidity",
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)